# 18.065 Problem Set 6 Solutions

## Problem 1 (10+10 points)

**(a)** In class, we showed that if $\tilde{y} = \bar{F} y, \tilde{a} = \bar{F} a, \tilde{x} = \bar{F} x$ where $F$ is the DFT matrix, then $y = a \circledast x$ (circular convolution) implies $\tilde{y} = \tilde{a} \, \mbox{.*} \, \tilde{x}$ (element-wise product).   Show that the reverse is also true: show that if $\tilde{y} = \tilde{a} \circledast \tilde{x}$, then $y = \alpha (a \, \mbox{.*} \, x) $ for some scale factor $\alpha = ???$. 

**(b)** Differentiating through convolutions, in class, involved the "reversal" permutation $R$:
$$
R\begin{pmatrix} x_0 \\ x_1 \\ x_2 \\ \vdots \\ x_{N-1} \end{pmatrix}
=\begin{pmatrix} x_0 \\ x_{N-1} \\ x_{N-2} \\ \vdots \\ x_1 \end{pmatrix}
$$
What happens when we combine this with a DFT, e.g. to apply the convolution theorem?  Show that $FRx = \bar{F}x$.

(This is illustrated by the code below: `fft(x)` computes $Fx$ and `bfft(x)` computes $\bar{F}x$ using the [FFTW.jl package](https://github.com/JuliaMath/FFTW.jl).)

In [1]:
using FFTW
R(x) = [x[begin]; x[end:-1:begin+1]] # compute R*x
x = rand(20)
fft(R(x)) ≈ bfft(x)

true

## Solutions:

**(a)** There are many ways to approach this problem, one approach is simply to re-trace our steps from class.

In class, we showed that the columns of $F$ are eigenvectors of cyclic convolutions, with the corresponding eigenvalues given by $\bar{F} a$.  Hence we obtained $a \circledast x = F (\bar{F}a \,\mbox{.*}) F^{-1} x =  \frac{1}{N} F (\bar{F}a \, \mbox{.*} \, \bar{F} x) = \bar{F}^{-1} (\tilde{a} \,\mbox{.*}\, \tilde{x})$.

Now, we want to show almost the same thing but we want to transform in the opposite direction.  That is, we want to replace $F$ with $F^{-1} = \bar{F}/N$ and vice versa.   If we proceed as in class, we can start by showing that the columns of $\bar{F}$ are eigenvectors of convolutions.   But our proof from class for $F$ only relied on the cyclic property $\omega_N^N = 1$!  The proof is completely unchanged if we replace $\omega_N$ with $\overline{\omega_N}$ (corresponding to replacing $F$ with $\bar{F}$)!

Hence we can simply quote the results above and replace $F$ with $\bar{F}$:
$$
\tilde{y} = \tilde{a} \circledast \tilde{x} = \bar{F} (F\tilde{a}\, \mbox{.*}) \bar{F}^{-1} \tilde{x} 
=  F^{-1} (F\tilde{a} \,\mbox{.*}\, F\tilde{x})
$$
Since $\tilde{x} = \bar{F} x$, that means $x = \bar{F}^{-1} \tilde{x} = F \tilde{x} / N$, i.e. $F\tilde{x} = Nx$.  Similarly for $a$.  Substituting that in, we obtain:
$$
\tilde{y} = N^2 F^{-1} (a \,\mbox{.*}\, x) \Longleftrightarrow \boxed{y = N (a \,\mbox{.*}\, x)}.
$$
which is the desired result with $\boxed{\alpha = N}$.

*Optional*: Since it's easy to get this algebra wrong, let's do a quick check of our result in Julia:

In [3]:
# brute-force circular convolution
function conv(a,x)
    N = length(a); N == length(x) || throw(DimensionMismatch())
    return [sum(i -> a[begin+i] * x[begin+mod(j-i, N)], 0:N-1) for j in 0:N-1]
end

N = 10
ã = randn(ComplexF64, N)
x̃ = randn(ComplexF64, N)
ỹ = conv(ã, x̃)
a = fft(ã)/N # Fã/N
x = fft(x̃)/N # Fx̃/N
y = fft(ỹ)/N # Fỹ/N
y ≈ N * (a .* x)

true

Hooray!  Math works!

**(b)**

If $y = Rx$, then $y_n = x_{-n}$ (with indices interpreted $\mod N$ as usual).  If $z = FRx = Fy$, then we have:
$$
z_m = \sum_{n=0}^{N-1} \omega_N^{mn} y_n = \sum_{n=0}^{N-1} \omega_N^{mn} x_{-n} = \sum_{k=0}^{N-1} \omega_N^{-mk} x_{k} = (\bar{F} x)_m
$$
where we simply performed a change of variables $k = -n$ in the sum.  Hence $z = FRx = \bar{F}x$ as desired.

## Problem 2 (10+10+10 points)

Think way back to grade school, when you learned to multiply two numbers $a_{N_a-1} \cdots a_2 a_1 a_0 \times x_{N_x-1} \cdots x_2 x_1 x_0$, where $a_k$ and $x_k$ are the **decimal digits** of two numbers with $N_a$ and $N_x$ digits, respectively.  (That is, $x_{N_x-1} \cdots x_2 x_1 x_0 = \sum_k {10}^k x_k$.)

**(a)** Show that, if you ignore carries (i.e. you allow "digits" $> 9$) then the *result* $ y_{N_y-1} \cdots y_2 y_1 y_0$ is in fact a **convolution** of the digits $y = a \ast x$, but *not* a cyclic convolution — there is no "wraparound".  (What is $N_y$?)

**(b)** Show that you can write your previous answer as a *cyclic* convolution simply by "padding" $a,x,y$ with zeros.  That is, if you add some zeros to each of them (in the right place) to extend them to a common length $N$ (what must $N$ be?), then $\mbox{pad}(y,N) = \mbox{pad}(a,N) \circledast \mbox{pad}(x,N)$.

**(c)** Write Julia code to compute
$$
\underbrace{94403147635990179114}_a \times \underbrace{33855185913241354461}_x \\
= \underbrace{3196036114011618584561027454963352927554}_y
$$
by:
* Implement the function `pad(x,N)` to zero-pad as in part (b)
* Implement a function `conv(a,x)` to perform a circular convolution using $Fx =$ `fft(x)` and $\bar{F}x =$ `bfft(x)`.  (You may want to call `round.(Int,real(y))` on the result to discard tiny roundoff errors, since the final values should be integers).
* Implement a function `carry(y)` to perform the carries: any "digits" larger than $10$ should have the tens place "carried" to the next digit etc.  (You may find the [`divrem` function](https://docs.julialang.org/en/v1/base/math/#Base.divrem) useful to divide by 10 and find a remainder.)
Check that you get the right answer: `bignum(carry(y)) == 3196036114011618584561027454963352927554`.

In [4]:
using FFTW

a = [4, 1, 1, 9, 7, 1, 0, 9, 9, 5, 3, 6, 7, 4, 1, 3, 0, 4, 4, 9]
x = [1, 6, 4, 4, 5, 3, 1, 4, 2, 3, 1, 9, 5, 8, 1, 5, 5, 8, 3, 3]

bignum(x) = evalpoly(BigInt(10), x)
@show bignum(a)
@show bignum(x)
@show bignum(a) * bignum(x)

bignum(a) = 94403147635990179114
bignum(x) = 33855185913241354461
bignum(a) * bignum(x) = 3196036114011618584561027454963352927554


3196036114011618584561027454963352927554

## Solutions:

**(a)** Think about the usual "long multiplication algorithm, omitting carries.  To get the 1's place of the output, i.e. the "digit" $y_0$, we simply multiply the 1's -places of the inputs: $y_0 = a_0 x_0$.   To get the 10's place of the ouptut, i.e. the "digit" $y_1$, we multiply the 1's place of $a$ with the 10's place of $x$ and vice versa, and add them, i.e. $y_1 = a_0 x_1 + a_1 x_0$.  And so forth, giving a pattern like:
$$
y_0 = a_0 x_0 \\
y_1 = a_0 x_1 + a_1 x_0 \\
y_2 = a_0 x_2 + a_1 x_1 + a_2 x_0 \\
\vdots \\
y_m = \sum_{n=0}^m a_n x_{m-n}
$$
which is exactly in the form of a linear convolution!  To make the sums look more uniform, we can equivalently think of each number as having an *infinite* number of digits, but most of the digits are zero (e.g. $x_k = 0$ for $k < 0$ or $k \ge N_x$, where the former correspond to "zeros after the decimal point" and the latter correspond to zeros before the most significant digit).  With this convention, we can simply  write:
$$
y_m = \sum_{n=-\infty}^\infty a_n x_{m-n}
$$

**(b)** We must pad with zeros to a length $N$ that is big enough that the circular "wrap around" of a cyclic convolution does not distort the result — the "wrapped around" digits must all be zeros.  Equivalently, since a convolution is a sequence of dot product where we "slide" $x$ past $a$ (or vice versa), the overlap should be zero before $x$ slides past the boundary.  The last possible overlap, corresponds to the most significant digit in the output $y$, which is $y_m$ for $m = N_a + N_x - 2$: the output (not including carries) has $N_a + N_x - 1$ digits.
In short, we simply need to pad to a length long enough to hold all the digits of the output, i.e. to any $\boxed{N \ge N_a + N_x - 1}$.

Be careful in part (c), however!  Once we include carries, this can introduce *additional* digits, so the carried result may eventually be a longer length.

**(c)** Our implementation and tests are as follows

In [22]:
# pad x with N-length(x) zeros of the same type
pad(x, N) = [x; zeros(eltype(x), N - length(x))]
    
# there are many ways to write this,
# since from problem 1 the convolution theorem takes many forms
conv(x,y) = ifft(fft(x) .* fft(y))
# or: conv(x,y) = bfft(fft(x) .* fft(y)) / length(x)
# or: conv(x,y) = fft(bfft(x) .* bfft(y)) / length(x)

# for integer vectors, we expect an integer result, but
# FFTs will give us slightly complex/noninteger results due
# to roundoff errors.  Let's correct this as suggested:
conv(x::AbstractVector{<:Integer}, y::AbstractVector{<:Integer}) =
    round.(promote_type(eltype(x),eltype(y)), real.(conv(float.(x), float.(y))))

conv (generic function with 2 methods)

Before we do any carries, let's check that our convolution and zero-padding are correct — evaluating the resulting "digits" as an integer using the `bignum` function from above should still work even if we have digits $> 10$.

In [23]:
Na = length(a)
Nx = length(x)

# any length ≥ Na + Nx - 1 should work.  let's try a few:
for N in (Na + Nx - 1) .+ (0:5)
    y = conv(pad(a, N), pad(x, N))
    @show bignum(y) == bignum(a) * bignum(x)
end

bignum(y) == bignum(a) * bignum(x) = true
bignum(y) == bignum(a) * bignum(x) = true
bignum(y) == bignum(a) * bignum(x) = true
bignum(y) == bignum(a) * bignum(x) = true
bignum(y) == bignum(a) * bignum(x) = true
bignum(y) == bignum(a) * bignum(x) = true


Now, let's implement the carry function.  The key step, for a digit $y_k = {}$ `y[k]`, is that we want to find the quotient $q$ and the remainder $r$ after dividing by 10, which can be accomplished in Julia by:
```jl
q, r = divrem(y[k], 10)
```
Then, the remainder is the *new k-th digit* ($y_k \to r$) and the quotient *gets carried* (added to $y_{k+1}$).   Note, however, that `y[k]` in this expression *includes any previous carries*.

In [29]:
function carry(y)
    c = similar(y, 0) # result array of the same type as y, of zero length
    push!(c, 0) # let's start off with a result of zero.
    for k = 1:length(y)
        q, r = divrem(y[k]+c[k], 10)
        c[k] = r # remainder = new digit (+ any previous carry)
        push!(c, q) # push quotient as the next digit
    end
    return c
end

carry (generic function with 1 method)

Let's try it:

In [32]:
N = Na + Nx - 1
y = carry(conv(pad(a, N), pad(x, N)))

@show y
@show y == digits(bignum(a) * bignum(x))
@show bignum(y) == bignum(a) * bignum(x)

y = [4, 5, 5, 7, 2, 9, 2, 5, 3, 3, 6, 9, 4, 5, 4, 7, 2, 0, 1, 6, 5, 4, 8, 5, 8, 1, 6, 1, 1, 0, 4, 1, 1, 6, 3, 0, 6, 9, 1, 3]
y == digits(bignum(a) * bignum(x)) = true
bignum(y) == bignum(a) * bignum(x) = true


true

Hooray, it's the correct digits $319603 \ldots 927554$ (starting with the least signifcant!) and passes all of our tests!

Note that the carried result is one digit longer than $N$ digits:

In [33]:
@show N
@show length(y)

N = 39
length(y) = 40


40